# Import Bokeh

Note: 
- the `ShowApp` function should take care of all proxy/jupyter hub problem
- change configuration (i.e. `USE_PUBLIC_IP` and `SSH_TUNNELS`) as needed

In [8]:
import os,sys,logging, urllib
import urllib.request
from pprint import pprint

# this may be dangerous, only for local testing/debugging
os.environ["BOKEH_ALLOW_WS_ORIGIN"]="*"

# if you want more debug info
os.environ["BOKEH_LOG_LEVEL"]="debug" 

import bokeh
import bokeh.io 
import bokeh.io.notebook 
import bokeh.models.widgets  
import bokeh.core.validation
import bokeh.plotting
import bokeh.core.validation.warnings
import bokeh.layouts

bokeh.core.validation.silence(bokeh.core.validation.warnings.EMPTY_LAYOUT, True)
bokeh.core.validation.silence(bokeh.core.validation.warnings.FIXED_SIZING_MODE,True)
bokeh.io.output_notebook()

# in JypyterLab/JupyterHub we need to tell what is the proxy url
# see https://docs.bokeh.org/en/3.0.3/docs/user_guide/output/jupyter.html
# example: 
USE_PUBLIC_IP=False

# change this if you need ssh-tunneling
# see https://github.com/sci-visus/OpenVisus/blob/master/docs/ssh-tunnels.md
SSH_TUNNELS=None

# //////////////////////////////////////////////////////////////////////////////////////
def ShowApp(app):
    
    if SSH_TUNNELS:
        notebook_port,bokeh_port=SSH_TUNNELS
        print(f"ShowApp, enabling ssh tunnels notebook_port={notebook_port} bokeh_port={bokeh_port}")
        bokeh.io.notebook.show_app(app, bokeh.io.notebook.curstate(), f"http://127.0.0.1:{notebook_port}", port = bokeh_port) 
        
    elif USE_PUBLIC_IP:
        
        # retrieve public IP (this is needed for front-end browser to reach bokeh server)
        public_ip = urllib.request.urlopen('https://ident.me').read().decode('utf8')
        print(f"public_ip={public_ip}")    
        
        if "JUPYTERHUB_SERVICE_PREFIX" in os.environ:

            def GetJupyterHubNotebookUrl(port):
                if port is None:
                    ret=public_ip
                    print(f"GetJupyterHubNotebookUrl port={port} returning {ret}")
                    return ret
                else:
                    ret=f"https://{public_ip}{os.environ['JUPYTERHUB_SERVICE_PREFIX']}proxy/{port}"
                    print(f"GetJupyterHubNotebookUrl port={port} returning {ret}")
                    return ret     

            bokeh.io.show(app, notebook_url=GetJupyterHubNotebookUrl)
            
        else:
            # need the port (TODO: test), I assume I will get a non-ambiguos/unique port
            import notebook.notebookapp
            ports=list(set([it['port'] for it in notebook.notebookapp.list_running_servers()]))
            assert(len(ports)==1)
            port=ports[0]
            notebook_url=f"{public_ip}:{port}" 
            print(f"bokeh.io.show(app, notebook_url='{notebook_url}')")
            bokeh.io.show(app, notebook_url=notebook_url)
    else:
        bokeh.io.show(app) 
      
# //////////////////////////////////////////////////////////////////////////////////////
def TestBokehApp(doc):
    fig = bokeh.plotting.figure(title="Multiple line example", x_axis_label="x", y_axis_label="y",height=200)
    fig.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5])
    grid=bokeh.layouts.grid(children=[fig], nrows=1, ncols=1, sizing_mode='stretch_width') 
    main_layout=bokeh.layouts.column(children=[],sizing_mode='stretch_width')
    main_layout.children.append(grid)  
    button = bokeh.models.widgets.Button(label="Bokeh is working", sizing_mode='stretch_width')
    button.on_click(lambda: print("Bokeh on_click event is working"))     
    main_layout.children.append(button)
    doc.add_root(main_layout)  

ShowApp(TestBokehApp)

AttributeError: module 'bokeh.core.validation.warnings' has no attribute 'EQUAL_SLIDER_START_END'

# Import OpenVisus

In [2]:
sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")
import OpenVisus as  ov
from OpenVisus.dashboards import Slice,Slices
os.environ["VISUS_NETSERVICE_VERBOSE"]="0"
print("OpenVisus imported")

OpenVisus imported


# Show single slice of a RGB 2D dataset (David)

In [3]:
def MyApp(doc, url=f"http://atlantis.sci.utah.edu/mod_visus?dataset=david_subsampled&cached=1"):
    db=ov.LoadDataset(url)    
    slice=Slice(doc,sizing_mode='stretch_width')
    slice.show_options=["palette","timestep","field","direction","offset"]
    slice.setDataset(db)
    slice.setDirection(2)
    slice.setPalette("Greys256", palette_range=(0,255))
    slice.setTimestep(db.getTime())
    slice.setField(db.getField().name)         
    doc.add_root(slice.layout)
ShowApp(MyApp)

# Single single slice of a gray 3D dataset (2kbit1)

In [4]:
def MyApp(doc, url=f"http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1&cached=1"):
    db=ov.LoadDataset(url)    
    slice=Slice(doc,sizing_mode='stretch_width')
    slice.show_options=["palette","field","direction","offset"]
    slice.setDataset(db)   
    slice.setDirection(2)
    slice.setPalette("Greys256", palette_range=(0,255))
    slice.setTimestep(db.getTime())
    slice.setField(db.getField().name)         
    doc.add_root(slice.layout)
ShowApp(MyApp)

# Show multiple slices of a RGB 2D dataset (David)

In [5]:
def MyApp(doc,url=f"http://atlantis.sci.utah.edu/mod_visus?dataset=david_subsampled&cached=1"):
    db=ov.LoadDataset(url) 
    slices=Slices(doc,sizing_mode='stretch_width')
    slices.setDataset(db)
    slices.setNumberOfViews(2)
    slices.setPalette("Greys256", palette_range=(0,255))
    slices.setTimestep(db.getTime())
    slices.setField(db.getField().name)         
    doc.add_root(slices.layout)
ShowApp(MyApp)

# Show multiple slices of a grayscale 3D dataset (2kbit1)

In [6]:
def MyApp(doc,url=f"http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1&cached=1"):
    db=ov.LoadDataset(url) 
    slices=Slices(doc,sizing_mode='stretch_width')
    slices.setDataset(db)
    slices.setNumberOfViews(2)
    slices.setPalette("Greys256", palette_range=(0,255))
    slices.setTimestep(db.getTime())
    slices.setField(db.getField().name)         
    doc.add_root(slices.layout)
ShowApp(MyApp)

ERROR:bokeh.core.validation.check:E-1021 (EQUAL_SLIDER_START_END): Slider 'start' and 'end' cannot be equal: Slider(id='1281', ...) with title Offset
ERROR:bokeh.core.validation.check:E-1021 (EQUAL_SLIDER_START_END): Slider 'start' and 'end' cannot be equal: Slider(id='1220', ...) with title Offset
ERROR:bokeh.core.validation.check:E-1021 (EQUAL_SLIDER_START_END): Slider 'start' and 'end' cannot be equal: Slider(id='1391', ...) with title Offset
ERROR:bokeh.core.validation.check:E-1021 (EQUAL_SLIDER_START_END): Slider 'start' and 'end' cannot be equal: Slider(id='1471', ...) with title Time
ERROR:bokeh.core.validation.check:E-1021 (EQUAL_SLIDER_START_END): Slider 'start' and 'end' cannot be equal: Slider(id='1557', ...) with title Time


Bokeh on_click event is working
Bokeh on_click event is working
